In [ ]:
#Importing Libraries
import pandas as pd 
import numpy as np

In [ ]:
#Datafile Import
df = pd.read_csv("online_retail_clean.csv")

In [68]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2021-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2021-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2021-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2021-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2021-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


# Cleaning & Preparation

In [69]:
df.shape

(824364, 9)

In [70]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
Revenue        0
dtype: int64

In [48]:
df['Customer ID'].count()

np.int64(824364)

In [54]:
# 1. Remove missing CustomerID
df = df.dropna(subset=['Customer ID'])

In [49]:
df.isnull().sum()

Invoice        0
StockCode      0
Description    0
Quantity       0
InvoiceDate    0
Price          0
Customer ID    0
Country        0
dtype: int64

In [55]:
# 4. Create Revenue column
df["Revenue"] = df["Quantity"] * df["Price"]

In [56]:
# 5. Convert InvoiceDate to datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])


In [58]:
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,Revenue
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,2009-12-01 07:45:00,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,2009-12-01 07:45:00,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,2009-12-01 07:45:00,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,2009-12-01 07:45:00,1.25,13085.0,United Kingdom,30.0


In [61]:
df.columns

Index(['Invoice', 'StockCode', 'Description', 'Quantity', 'InvoiceDate',
       'Price', 'Customer ID', 'Country', 'Revenue'],
      dtype='object')

In [76]:
# 5. Convert InvoiceDate to datetime
df["InvoiceDate"] = pd.to_datetime(df["InvoiceDate"])

In [65]:
df['InvoiceDate'].count

<bound method Series.count of 0         2021-12-01 07:45:00
1         2021-12-01 07:45:00
2         2021-12-01 07:45:00
3         2021-12-01 07:45:00
4         2021-12-01 07:45:00
                  ...        
1067366   2023-12-09 12:50:00
1067367   2023-12-09 12:50:00
1067368   2023-12-09 12:50:00
1067369   2023-12-09 12:50:00
1067370   2023-12-09 12:50:00
Name: InvoiceDate, Length: 824364, dtype: datetime64[ns]>

In [71]:
# -----------------------------
# 1. Customer Acquisition Channel (synthetic)
# -----------------------------
channels = ["Paid Search", "Social", "Organic", "Referral", "Email"]
probs = [0.25, 0.20, 0.30, 0.15, 0.10]  # probabilities

# Create mapping: each CustomerID gets one acquisition channel
unique_customers = df["Customer ID"].unique()
cust_channels = np.random.choice(channels, size=len(unique_customers), p=probs)
channel_map = dict(zip(unique_customers, cust_channels))

df["Channel"] = df["Customer ID"].map(channel_map)


In [72]:
# -----------------------------
# 2. Customer Acquisition Cost (CAC) by Channel
# -----------------------------
cac_map = {
    "Paid Search": 15,
    "Social": 12,
    "Organic": 5,
    "Referral": 8,
    "Email": 4
}
df["CAC"] = df["Channel"].map(cac_map)


In [73]:
# -----------------------------
# 3. Loyalty Flag (synthetic ~25%)
# -----------------------------
np.random.seed(42)  # reproducibility
cust_loyalty = np.random.choice([0, 1], size=len(unique_customers), p=[0.75, 0.25])
loyalty_map = dict(zip(unique_customers, cust_loyalty))
df["Loyalty"] = df["Customer ID"].map(loyalty_map)


In [77]:
# -----------------------------
# 4. Seasonality Index (month multipliers)
# -----------------------------
seasonality = {
    1: 0.85,  # Jan dip
    2: 0.90,
    3: 0.95,
    4: 1.00,
    5: 1.05,
    6: 1.00,
    7: 0.95,
    8: 0.98,
    9: 1.05,
    10: 1.10,
    11: 1.25, # Nov uplift
    12: 1.60  # Dec peak
}

df["Month"] = df["InvoiceDate"].dt.month
df["SeasonalityIndex"] = df["Month"].map(seasonality)


In [78]:
df.to_csv("online_retail_enriched.csv", index=False)

print("✅ Enriched dataset saved as 'online_retail_enriched.csv'")
print(df.head())

✅ Enriched dataset saved as 'online_retail_enriched.csv'
  Invoice StockCode                          Description  Quantity  \
0  489434     85048  15CM CHRISTMAS GLASS BALL 20 LIGHTS        12   
1  489434    79323P                   PINK CHERRY LIGHTS        12   
2  489434    79323W                  WHITE CHERRY LIGHTS        12   
3  489434     22041         RECORD FRAME 7" SINGLE SIZE         48   
4  489434     21232       STRAWBERRY CERAMIC TRINKET BOX        24   

          InvoiceDate  Price  Customer ID         Country  Revenue Channel  \
0 2021-12-01 07:45:00   6.95      13085.0  United Kingdom     83.4   Email   
1 2021-12-01 07:45:00   6.75      13085.0  United Kingdom     81.0   Email   
2 2021-12-01 07:45:00   6.75      13085.0  United Kingdom     81.0   Email   
3 2021-12-01 07:45:00   2.10      13085.0  United Kingdom    100.8   Email   
4 2021-12-01 07:45:00   1.25      13085.0  United Kingdom     30.0   Email   

   CAC  Loyalty  Month  SeasonalityIndex  
0    4    